<a href="https://colab.research.google.com/github/aleks-haksly/KarpovCources_Hard_DA_advanced/blob/main/04%20-%20ML/02_ml_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import userdata

## Задание 1. Спецификация (1/2)


Зафиксируйте сумму выручки по месяцу каждого года. Далее мы будем использовать это как внешнюю информацию.

По-хорошему, в проде такую информацию стоит обновлять каждый месяц - почти наверняка даже самая простая инфраструктура в компании позволит это сделать с наступлением нового месяца.

Создайте из полученных данных словарь `quasi_external` такого вида, он пригодится нам позже:

`{'2021-02': сумма выручки,
 '2021-03': сумма выручки,
 '2021-04': сумма выручки,
 '2021-05': сумма выручки
...}`


Какая сумма транзакций получилась за июнь 2022 года?

In [ ]:
data = pd.read_csv("/content/premium_by_passports.csv", parse_dates=["payment_date", "created_at"])

In [ ]:
data["payment_month"] = data["payment_date"].dt.strftime("%Y-%m")

In [ ]:
quasi_external = data.groupby("payment_month").revenue.sum().to_dict()

In [ ]:
print(f"Сумма транзакций за июнь 2022 = {quasi_external.get('2022-06')}")

Сумма транзакций за июнь 2022 = 6537800


## Задание 1. Спецификация (2/2)


Нам необходимо получить табличку, в которой строкой будет пара объект-таргет вида (`passport_id` - сумма платежей через месяц).

Во-первых, почистим платежи, которые были совершены клиентом через 30 дней после онбординга.

Во-вторых, необходимо сгруппировать оставшиеся платежи в одну сумму - именно эту метрику мы и будем предсказывать.

Группировку нужно делать по признакам `'type'`, `'passport_id'`, `'created_at'`

P.S. заметьте, что у многих `passport_id` в каких-то платежах пропущена категориальная информация (`user_type_name` и `user_type_cars_name`), а в каких-то нет. Тем не менее, по устройству базы мы понимаем, что эта информация уникальна, и в данной задаче не меняется во времени. Нужно подумать над тем, как обработать категориальные колонки таким образом, чтобы у всех платежей одного клиента там была указана верная информация, если она указана в любом другом его платеже.
В pandas можно использовать функцию агрегации first

Какое количество строк получилось в таблице?

In [ ]:
data_filtered = data[data["payment_date"] <= data["created_at"] + pd.Timedelta(days=31)]
#data_filtered = data[(data['payment_date'] - data['created_at']).dt.days <= 30]

In [ ]:
df = data_filtered.groupby(["passport_id", "created_at", "type"]).agg({"user_type_name": 'first', "user_type_cars_name": 'first', "revenue": 'sum'})

In [ ]:
print(f"количество строк в таблице {df.shape[0]}")

количество строк в таблице 8770


## Задание 2. Обработка данных (2/5) ##


Определите значения 2.5 и 97.5 квантилей для таргета. Удалите из данных все записи, значения целевой переменной которых находятся за пределами указанных квантилей.

Введите полученные значения через запятую и пробел - сначала значение 2.5 квантиля, после значение 97,5 квантиля для таргетной переменной. Округлите значения до целого.

In [ ]:
df_no_outliers = df[(df["revenue"] > df["revenue"].quantile(0.025)) & (df["revenue"] < df["revenue"].quantile(0.975))]

In [ ]:
df["revenue"].quantile(0.025).round(), df["revenue"].quantile(0.975).round()

(685.0, 5188.0)

## Задание 2. Обработка данных (3/5)


Изучите, есть ли в данных пропуски. В каких колонках вы их обнаружили?

In [ ]:
# Пропуски есть в колонках:
df_no_outliers.reset_index().isna().sum().where(lambda x : x!=0).dropna().index.tolist()

['user_type_name', 'user_type_cars_name']

## Задание 2. Обработка данных (4/5)


Есть ли строки, в которых половина или более колонок пустые? Как думаете, такое возможно?

## Задание 2. Обработка данных (5/5)
Напишите кастомный класс `FilteringSelector`, который оставляет только те колонки, в которых не более `t=40%` пропусков. Сделайте это по аналогии с практической частью урока. В этом случае в качестве параметра инициализации можно выбрать порог t, а можно опустить и зашить внутрь кода класса.

В уроке мы делали фильтрацию фичей внутри кастомного трансформера, в котором, в том числе, генерили новые признаки. Уже обсуждали, что так лучше на практике не делать, и разные части выносить в разные классы и функции. Фильтрацию данных - в одно место, создание новых фичей - в другое.

Не забудьте обернуть `FilteringSelector` в `ColumnTransformer`, как мы это делали в уроке, инициализируя переменную `col_selector`.

Вставьте в поле ниже только реализацию класса `FilteringSelector`, переменную `col_selector` сдавать не нужно. Не забудьте добавить импорт pandas

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer

In [ ]:
class FilteringSelector :

    def __init__(self, threshold=0.4):
        self.threshold = threshold


    def __call__(self, df):
        if not hasattr(df, "iloc"):
            raise ValueError(
                "make_column_selector can only be applied to pandas dataframes"
            )

        return df.isna().mean().where(lambda x : x < self.threshold).dropna().index.tolist()

In [ ]:
col_selector = ColumnTransformer(
    transformers=[
        ('ThresholdFilter', 'passthrough', FilteringSelector())
    ],
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas

In [ ]:
X_ = df_no_outliers.reset_index().drop(columns=["revenue", "type"])
y_ = df_no_outliers.reset_index()["revenue"]

In [ ]:
col_selector.fit_transform(X_, y_).head()

,passport_id,created_at,user_type_name
0,140366939,2021-01-04 11:09:27,simple_user
1,140386549,2021-01-06 16:54:42,simple_user
2,140387667,2021-01-06 19:07:18,simple_user
3,140390659,2021-01-07 01:08:09,simple_user
4,140391889,2021-01-07 11:29:07,simple_user


## Задание 3. Трансформер
  

Напишите класс `AddColumnsTransformer`, который делает следующие действия:

1) Выделяет из даты создания профиля квартал - колонка `Quarter` в формате `'2023Q3'`

2) Из созданного ранее словаря формирует фичу `last_month_pmts`, которая представляет собой сумму транзакций в предыдущем месяце, используя внешние данные. Если данных о платежах за предыдущий месяц нет, то возьмите среднее по всем доступным месяцам

3) Удаляет колонки-ключи: passport_id  (названия этой колонки можно захардкодить) и created_at

У такого класса должно быть всего два параметра:

1) created_at_column, содержащий название колонки с датой создания профиля

2) payments_by_month переменная, содержащая словарь с информацией о сумме платежей по месяцам из выборки, который мы создавали ранее

Вставьте в поле код класса AddColumnsTransformer, не забудьте необходимые импорты

Словарь с суммами выручки по месяцу каждого года уже загружен в LMS в переменную quasi_external

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class AddColumnsTransformer(BaseEstimator, TransformerMixin):
  def __init__(
        self,
        created_at_column='created_at',
        payments_by_month=None
    ):
        self.created_at_column = created_at_column
        self.payments_by_month = payments_by_month
  def fit(self, X, y):

        if not hasattr(X, "iloc"):
            raise ValueError(
                "CustomTransformer can only be applied to pandas dataframes in X argument"
            )

        self.payments_by_month_mean = np.mean(list(self.payments_by_month.values()))
        self.columns_to_drop = ['passport_id', self.created_at_column]

        return self

  def transform(self, X):
        if not hasattr(X, "iloc"):
            raise ValueError(
                "CustomTransformer can only be applied to pandas dataframes in X argument"
            )
        X_copy= X.copy()

        # Выделяет из даты создания профиля квартал - колонка Quarter в формате '2023Q3'
        X_copy['Quarter'] = X_copy[self.created_at_column].dt.to_period('Q').astype(str)

        # Из созданного ранее словаря формирует фичу last_month_pmts, которая представляет собой сумму транзакций в предыдущем месяце, используя внешние данные.
        X_copy['last_month_pmts'] = X_copy.created_at.apply(lambda x: self.payments_by_month.get((x - pd.offsets.MonthEnd()).strftime("%Y-%m"), self.payments_by_month_mean))

        # Удаляет колонки-ключи: passport_id  (названия этой колонки можно захардкодить) и created_at
        X_copy = X_copy.drop(columns=self.columns_to_drop, axis=1)

        return X_copy


In [ ]:
CustomTransformer = AddColumnsTransformer(created_at_column='created_at', payments_by_month=quasi_external)

In [ ]:
CustomTransformer.fit_transform(X_, y_)

,user_type_name,user_type_cars_name,Quarter,last_month_pmts
0,simple_user,None,2021Q1,3.114500e+06
1,simple_user,None,2021Q1,3.114500e+06
2,simple_user,None,2021Q1,3.114500e+06
3,simple_user,cars_simple,2021Q1,3.114500e+06
4,simple_user,cars_simple,2021Q1,3.114500e+06
...,...,...,...,...
8099,simple_user,None,2023Q1,4.635815e+06
8100,profi,None,2023Q1,4.635815e+06
8101,simple_user,None,2023Q1,4.635815e+06
8102,simple_user,cars_simple,2023Q1,4.635815e+06


## Задание 4. Пайплайн обработки данных
Подготовьте `final_process_pipe`, который будет состоять из следующих шагов:

1) Применение кастомного селектора для автоматического отбора колонок без большого числа пропусков

2) Создание новых фичей с помощью кастомного трансформера

3) Заполнение пропусков средним/модой у вещественных/категориальных колонок соответственно. Можно взять код как в практике, переменную `col_imputer`

4) `MeanTargetEcnoder` для всех категориальных фичей. Можно взять код как в практике и оставить только `MTE` (переменная `col_transformer_with_selector` из ноутбука). Важно: установите параметр `shuffle=False` в `TargetEncoder` для воспроизводимости результатов!

5) `StandardScaler` на оставшийся датафрейм - степ `num_scaler` из ноутбука

Сохраните получившийся датафрейм в файл `CSV` с параметром `index=False`

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
X_ = df_no_outliers.reset_index().drop(columns=["revenue", "type"])
y_ = df_no_outliers.reset_index()["revenue"]

In [ ]:
### Заполнение пропусков средним у numeric columns
### Заполнение пропусков модой у categorical columns
col_imputer = ColumnTransformer(
    transformers=[
        ('impute_num', SimpleImputer(strategy='mean'), selector(dtype_include="number")),
        ('impute_cat', SimpleImputer(strategy='most_frequent'), selector(dtype_exclude="number")) #missing_values = None
    ],
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas

In [ ]:
col_transformer_with_selector = ColumnTransformer(
    transformers=[
        ('MeanTargetEncoder', TargetEncoder(target_type="continuous", shuffle=False), selector(dtype_exclude="number"))
    ],
    remainder='passthrough',          # Это чтобы не дропнуть колонки, которых трансформер не касается
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas

In [ ]:
num_scaler = ColumnTransformer(
    transformers=[
        ('StandardScaler', StandardScaler(), selector(dtype_include='number'))
    ],
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
final_process_pipe = Pipeline(
    steps=[
        ('col_selector', col_selector),
        ('CustomTransformer', CustomTransformer),
        ('col_imputer', col_imputer),
        ('col_transformer_with_selector', col_transformer_with_selector),
        ("num_scaler", num_scaler)

    ]
)

In [ ]:
output = final_process_pipe.fit_transform(X_, y_)

In [ ]:
output.isna().sum()

user_type_name     0
Quarter            0
last_month_pmts    0
dtype: int64

In [ ]:
output.to_csv("output.csv", index=False)

## Задание 5. Валидация


Подготовим `splitter` для валидации данных

Воспользуйтесь стратегией `TimeSeriesSplit`. Не забудьте отсортировать датасет по колонке `created_at`, если это еще не сделано.

Установите `n_splits=4`, остальные параметры оставьте по дефолту.

Вставьте код переменной `splitter` ниже

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
splitter = TimeSeriesSplit(
    n_splits=4,
    max_train_size=None, # <--- Максимальный размер трейна, можно ограничить
    test_size=None,      # <--- По дефолту, максимально доступный размер теста
    gap=0                # <--- Отступ от конца train части
)

In [ ]:
X_sorted = X_.sort_values(by="created_at")

In [ ]:
print(splitter)
print(splitter.split(X_sorted))

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=4, test_size=None)
<generator object TimeSeriesSplit.split at 0x7f45cc755d90>


## Задание 6. Обучение и выбор модели (1/5)


Для классической LR попробуем модель без и с свободным коэффициентом β. Ключевая метрика - MAE.

Выберите из этих этих двух моделей ту, которая дает лучшее качество.

Введите среднее качество на валидации лучшей модели на тестовой выборке, округленное до целых:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [ ]:
# сортируем первоначальный dataframe по датам
df_no_outliers_sorted = df_no_outliers.sort_values(by="created_at").copy()

In [ ]:
# выделяем в нем X и y
X = df_no_outliers_sorted.reset_index().drop(columns=["revenue", "type"], axis=1)
y = df_no_outliers_sorted.reset_index()["revenue"]

In [ ]:
# формируем окончательный pipeline с добавлением регрессии без подбора свободного члена (intercept)
final_process_pipe_model = Pipeline(
    steps=[
        ('col_selector', col_selector),
        ('CustomTransformer', CustomTransformer),
        ('col_imputer', col_imputer),
        ('col_transformer_with_selector', col_transformer_with_selector),
        ("num_scaler", num_scaler),
        ('simple_model', LinearRegression(fit_intercept=False))

    ]
)

In [ ]:
# делаем кросс валидацию
cv_result = cross_validate(final_process_pipe_model,
                           X, y,
                           scoring='neg_mean_absolute_error',
                           cv=splitter, return_train_score=True)

cv_result

{'fit_time': array([0.11757064, 0.15087748, 0.19764638, 0.30150151]),
 'score_time': array([0.08037043, 0.07270646, 0.1110034 , 0.08625817]),
 'test_score': array([-1339.83240433, -1447.9918129 , -1350.41510398, -1481.99800209]),
 'train_score': array([-1422.36607896, -1364.31673965, -1367.56412147, -1368.5304439 ])}

In [ ]:
print(f"средняя MAE для регрессии без свободного коэффициента = {-cv_result['test_score'].mean():.0f}")

средняя MAE для регрессии без свободного коэффициента = 1405


In [ ]:
final_process_pipe_model_intercept = Pipeline(
    steps=[
        ('col_selector', col_selector),
        ('CustomTransformer', CustomTransformer),
        ('col_imputer', col_imputer),
        ('col_transformer_with_selector', col_transformer_with_selector),
        ("num_scaler", num_scaler),
        ('simple_model', LinearRegression(fit_intercept=True))

    ]
)

In [ ]:
cv_result_intercept = cross_validate(final_process_pipe_model_intercept,
                           X, y,
                           scoring='neg_mean_absolute_error',
                           cv=splitter, return_train_score=True)

cv_result_intercept

{'fit_time': array([0.1301198 , 0.16861081, 0.46086788, 0.30317354]),
 'score_time': array([0.09326887, 0.07266998, 0.09322166, 0.07309675]),
 'test_score': array([-608.93567918, -626.394523  , -617.9545887 , -644.73160992]),
 'train_score': array([-597.47556785, -594.86672221, -611.95353918, -612.86584067])}

In [ ]:
print(f"средняя MAE для регрессии со свободным коэффициентом = {-cv_result_intercept['test_score'].mean():.0f}")

средняя MAE для регрессии со свободным коэффициентом = 625


##  Наглядный способ

In [ ]:
for fold in splitter.split(X):
  print(fold[0], fold[1])

[   0    1    2 ... 1621 1622 1623] [1624 1625 1626 ... 3241 3242 3243]
[   0    1    2 ... 3241 3242 3243] [3244 3245 3246 ... 4861 4862 4863]
[   0    1    2 ... 4861 4862 4863] [4864 4865 4866 ... 6481 6482 6483]
[   0    1    2 ... 6481 6482 6483] [6484 6485 6486 ... 8101 8102 8103]


In [ ]:
for fold in splitter.split(X):
    train_index, val_index = fold[0], fold[1]

    X_to_train = X[X.index.isin(train_index)]
    X_to_val = X[X.index.isin(val_index)]

    y_to_train = y[y.index.isin(train_index)]
    y_to_val = y[y.index.isin(val_index)]

    eval_instance = final_process_pipe_model_intercept
    eval_instance.fit(X_to_train, y_to_train)

    pred = eval_instance.predict(X_to_val)
    error = (np.abs(pred - y_to_val)).mean()

    print(error)

608.9356791844457
626.3945230005204
617.954588704101
644.7316099219511


## Задание 6. Обучение и выбор модели (2/5)


Для `Lasso` и `Ridge` сетка по параметрам `alpha` и `max_iter` в массивах np.`linspace(start=0.1, stop=10000, num=20)` и `(100, 1000)`, соответственно.

Найдите набор параметров, который оказался наилучшим для `Lasso-регрессии`.

Введите значение `alpha`, округленное до одного знака после запятой:

In [ ]:
inal_process_pipe = Pipeline(
    [
        ('col_selector', col_selector),
        ('CustomTransformer', CustomTransformer),
        ('col_imputer', col_imputer),
        ('col_transformer_with_selector', col_transformer_with_selector),
        ("num_scaler", num_scaler)
    ]
)

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
### Сгенерируем Pipeline'ы с Lasso, Ridge Estimators

lasso_pipe = Pipeline(
    [
        ('all_preprocess', final_process_pipe),
        ('simple_model', Lasso())
    ]
)

ridge_pipe = Pipeline(
    [
        ('all_preprocess', final_process_pipe),
        ('simple_model', Ridge())
    ]
)

In [ ]:
param_grid = {
    "simple_model__alpha": np.linspace(start=0.1, stop=10000, num=20),
    "simple_model__max_iter": [100, 1000]
}

In [ ]:
for model in [ridge_pipe, lasso_pipe]:
    ### Передадим в GridSearchCV
    search = GridSearchCV(model, param_grid,
                          cv=splitter, scoring='neg_mean_absolute_error')

    search.fit(X, y)

    print(f"Best parameter (CV score={search.best_score_:.5f}):")
    print(search.best_params_)

Best parameter (CV score=-624.50444):
{'simple_model__alpha': 0.1, 'simple_model__max_iter': 100}
Best parameter (CV score=-624.50445):
{'simple_model__alpha': 0.1, 'simple_model__max_iter': 100}


In [ ]:
print(f"для Lasso-регрессии лучшая alpha = {search.best_params_['simple_model__alpha']}")

для Lasso-регрессии лучшая alpha = 0.1
